In [1]:
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('ratings.txt', delimiter='\t', header=0)
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [3]:
print("결측치 제거 전 데이터셋 길이:", len(df))
print("각 컬럼별 결측치 개수: ", df.isnull().sum())
df = df.dropna() # 결측치가 있는 행 제거
print("결측치 제거 후 데이터셋 길이:", len(df), "\n")

결측치 제거 전 데이터셋 길이: 200000
각 컬럼별 결측치 개수:  id          0
document    8
label       0
dtype: int64
결측치 제거 후 데이터셋 길이: 199992 



In [4]:
okt = Okt()
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

In [5]:
def text_preprocess(text):
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', text) # 한글과 공백만 남김
    tokens = okt.morphs(text, stem=True) # 형태소 분석 및 어간 추출
    tokens = [word for word in tokens if word not in stopwords] # 불용어 제거
    return ' '.join(tokens)

In [6]:
sample_text = df['document'].iloc[0]
print("원문:", sample_text)
print("전처리:", text_preprocess(sample_text), "\n")

원문: 어릴때보고 지금다시봐도 재밌어요ㅋㅋ
전처리: 어리다 때 보고 지금 다시 보다 재밌다 ㅋㅋ 



In [16]:
df['document2'] = df['document'].apply(text_preprocess)
df.to_csv('rating.csv', index=False, encoding='utf-8')
df.head()

,id,document,label,document2
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1,어리다 때 보고 지금 다시 보다 재밌다 ㅋㅋ
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1,디자인 을 배우다 학생 외국 디자이너 그 일군 전통 을 통해 발전 문화 산업 부럽다...
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1,폴리스스토리 시리즈 부터 뉴 까지 버리다 하나 없다 최고
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1,오다 연기 진짜 개 쩔다 지루하다 생각 몰입 보다 그렇다 이렇다 진짜 영화 지
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1,안개 자욱하다 밤하늘 뜨다 있다 초승달 같다 영화


In [8]:
df = pd.read_csv('rating.csv', delimiter=',')
df.head()

,id,document,label,document2
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1,어리다 때 보고 지금 다시 보다 재밌다 ㅋㅋ
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1,디자인 을 배우다 학생 외국 디자이너 그 일군 전통 을 통해 발전 문화 산업 부럽다...
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1,폴리스스토리 시리즈 부터 뉴 까지 버리다 하나 없다 최고
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1,오다 연기 진짜 개 쩔다 지루하다 생각 몰입 보다 그렇다 이렇다 진짜 영화 지
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1,안개 자욱하다 밤하늘 뜨다 있다 초승달 같다 영화


In [17]:
X = df['document2']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=456)

In [18]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print("학습용 데이터 :", X_train_vec.shape[0], ", 테스트용 데이터 :", X_test_vec.shape[0], "\n")


학습용 데이터 : 159993 , 테스트용 데이터 : 39999 



In [19]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_vec, y_train)
y_pred_knn = knn.predict(X_test_vec)
acc_knn = accuracy_score(y_test, y_pred_knn)
print(f"KNN 모델 정확도: {acc_knn}")


KNN 모델 정확도: 0.7272181804545114


In [20]:
print(f"best k : {knn.get_params()['n_neighbors']}")

best k : 3


In [21]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_vec, y_train)
y_pred_lr = lr.predict(X_test_vec)
acc_lr = accuracy_score(y_test, y_pred_lr)
print(f"로지스틱 회귀 모델 정확도: {acc_lr}")

로지스틱 회귀 모델 정확도: 0.8272456811420286


In [22]:
new_review = input("새로운 리뷰를 입력하세요: ") # : 이 영화 그저그럼. 추천 안해요
new_review_processed = text_preprocess(new_review)
new_review_vec = vectorizer.transform([new_review_processed])

In [23]:
pred_knn = knn.predict(new_review_vec)[0]
pred_lr = lr.predict(new_review_vec)[0]

In [14]:
def label_to_sentiment(label):
    return "긍정" if label == 1 else "부정"

print("KNN 모델 예측 결과:", label_to_sentiment(pred_knn))
print("로지스틱 회귀 모델 예측 결과:", label_to_sentiment(pred_lr))

KNN 모델 예측 결과: 긍정
로지스틱 회귀 모델 예측 결과: 긍정
